In [1]:
import os
import pandas as pd
import csv
import shutil
import numpy as np
import datetime

# Section 1

Se crean las carpetas en blanco para guardar los nuevos archivos modificados

In [2]:
#Crear carpetas en blanco

dir = "datos_Modificados"
if not os.path.exists(dir):
    os.makedirs(dir)

pollutants = ["CO", "NO", "NO2", "NOX", "O3", "PM2.5", "PM10", "RH", "SO2", "TMP", "WDR", "WSP"]
for pollutant in pollutants:
    pollutant_path = os.path.join(dir, pollutant)
    print(pollutant_path)
    os.makedirs(pollutant_path)
    year = 2024
    for y in range(1):
        year_path = os.path.join(pollutant_path, str(year+y))
        os.makedirs(year_path)
        print(year, year_path, y)



datos_Modificados\CO
2024 datos_Modificados\CO\2024 0
datos_Modificados\NO
2024 datos_Modificados\NO\2024 0
datos_Modificados\NO2
2024 datos_Modificados\NO2\2024 0
datos_Modificados\NOX
2024 datos_Modificados\NOX\2024 0
datos_Modificados\O3
2024 datos_Modificados\O3\2024 0
datos_Modificados\PM2.5
2024 datos_Modificados\PM2.5\2024 0
datos_Modificados\PM10
2024 datos_Modificados\PM10\2024 0
datos_Modificados\RH
2024 datos_Modificados\RH\2024 0
datos_Modificados\SO2
2024 datos_Modificados\SO2\2024 0
datos_Modificados\TMP
2024 datos_Modificados\TMP\2024 0
datos_Modificados\WDR
2024 datos_Modificados\WDR\2024 0
datos_Modificados\WSP
2024 datos_Modificados\WSP\2024 0


# Section 2

Los archivos descargados de la página aire y salud tiene una fila al inicio y dos al final que no pertenecen a la tabla de valores (el título e información pareciera de html),
por lo que se eliminan para que se puedan leer como dataframes con pandas, de otra forma causa error

In [3]:
def delete_rows(dir, new_dir, file_path):
    filas = []
    dataframe = pd.read_csv(file_path)
    num_rows = len(dataframe)
    print(num_rows, file_path)
    with open(file_path, 'r', newline='', encoding='utf-8') as archivo_csv:
        lector_csv = csv.reader(archivo_csv)
        for row in lector_csv:
            if "Promedio" in row[0]:
                print(row[0])    
            elif "<br" in row[0]:
                print(row[0])  
            elif "<b" in row[0]:
                print(row[0])          
            else:
                filas.append(row)


    subpath = file_path[len(dir):len(file_path)]
    new_path = os.path.join(new_dir, subpath)
    
    with open(new_path, 'w', newline='', encoding='utf-8') as archivo_csv_nuevo:
        escritor_csv = csv.writer(archivo_csv_nuevo)
        escritor_csv.writerows(filas)

    dataframe = pd.read_csv(new_path)
    num_rows = len(dataframe)
    print(num_rows)
    return dataframe, new_path


Se cambia el nombre de todos los archivos descargados (todos dicen Promedio_contaminanteX), por el nombre del contaminante_el mes_año

In [4]:
dir = "datos_Modificados/"
#dir = "Datos Aire y Salud/"
new_dir = "datos_Modificados/"


for actual_folder, folders, files in os.walk(dir):
    for file_name in files:
        file_path = os.path.join(actual_folder, file_name)
        if file_path.endswith(".csv"):
            sub_folders = file_path.split("/")
            sub_folders = sub_folders[len(sub_folders)-1].split("\\")
            variable = sub_folders[len(sub_folders)-3]
            year = sub_folders[len(sub_folders)-2]
            print(file_path, variable, year)

            try:
                dataframe, new_path = delete_rows(dir, new_dir, file_path)
            except:                
                subpath = file_path[len(dir):len(file_path)]
                path = os.path.join(new_dir, subpath)
                new_path = os.path.split(path)
                new_path = new_path[0]
                new_path = shutil.copy(file_path, new_path)
                dataframe = pd.read_csv(new_path)
            if file_name.startswith("Promedio"):
                months = {"01":"JAN", "02":"FEB", "03":"MAR", "04":"APR", "05":"MAY", "06":"JUN", "07":"JUL", "08":"AUG", "09":"SEP", "10":"OCT", "11":"NOV", "12":"DEC"}
                date = dataframe.iloc[0].Fecha
                num_month = date.split("-")
                month = months[num_month[1]]

                new_name = os.path.join(new_dir,variable)
                new_name = os.path.join(new_name, year)
                name = variable + "_" +month+"_"+year + ".csv"
                new_name = os.path.join(new_name, name)
                os.rename(new_path,new_name)


datos_Modificados/CO\2024\Promedio horarios de co (1).csv CO 2024
745 datos_Modificados/CO\2024\Promedio horarios de co (1).csv
Promedio horarios de coPromedio horarios de co
744
datos_Modificados/CO\2024\Promedio horarios de co (3).csv CO 2024
745 datos_Modificados/CO\2024\Promedio horarios de co (3).csv
Promedio horarios de coPromedio horarios de co
744
datos_Modificados/CO\2024\Promedio horarios de co (4).csv CO 2024
714 datos_Modificados/CO\2024\Promedio horarios de co (4).csv
Promedio horarios de coPromedio horarios de co
713
datos_Modificados/CO\2024\Promedio horarios de co.csv CO 2024
721 datos_Modificados/CO\2024\Promedio horarios de co.csv
Promedio horarios de coPromedio horarios de co
720
datos_Modificados/NO\2024\Promedio horarios de no (1).csv NO 2024
745 datos_Modificados/NO\2024\Promedio horarios de no (1).csv
Promedio horarios de noPromedio horarios de no
744
datos_Modificados/NO\2024\Promedio horarios de no (3).csv NO 2024
745 datos_Modificados/NO\2024\Promedio horarios

Se juntan POR CONTAMINANTE en un solo dataframe los datos de todos los meses y todos los año

In [5]:
new_dir = "datos_Modificados"
files_path = "Archivos_contaminantes_juntas"

if not os.path.exists(files_path):
    os.makedirs(files_path)
    
CO_dataframes = []
SO2_dataframes = []
NO_dataframes = []
NOX_dataframes = []
NO2_dataframes = []
PM10_dataframes = []
PM25_dataframes = []
O3_dataframes = []
RH_dataframes = []
WDR_dataframes = []
WSP_dataframes = []
TMP_dataframes = []


for actual_folder, folders, files in os.walk(new_dir):
    for file_name in files:
        file_path = os.path.join(actual_folder, file_name)
        df = pd.read_csv(file_path)
        if "CO" in file_name:
            CO_dataframes.append(df)
        elif "NOX" in file_name:
            NOX_dataframes.append(df)
        elif "NO2" in file_name:
            NO2_dataframes.append(df)
        elif "NO" in file_name:
            NO_dataframes.append(df)
        elif "O3" in file_name:
            O3_dataframes.append(df)
        elif "PM10" in file_name:
            PM10_dataframes.append(df)
        elif "PM2.5" in file_name:
            PM25_dataframes.append(df)
        elif "TMP" in file_name:
            TMP_dataframes.append(df)
        elif "WSP" in file_name:
            WSP_dataframes.append(df)
        elif "WDR" in file_name:
            WDR_dataframes.append(df)
        elif "RH" in file_name:
            RH_dataframes.append(df)
        elif "SO2" in file_name:
            SO2_dataframes.append(df)

df_final_CO = pd.concat(CO_dataframes, ignore_index=True, sort=False)
df_final_NO = pd.concat(NO_dataframes, ignore_index=True, sort=False)
df_final_NO2 = pd.concat(NO2_dataframes, ignore_index=True, sort=False)
df_final_NOX = pd.concat(NOX_dataframes, ignore_index=True, sort=False)
df_final_SO2 = pd.concat(SO2_dataframes, ignore_index=True, sort=False)
df_final_O3 = pd.concat(O3_dataframes, ignore_index=True, sort=False)
df_final_PM10 = pd.concat(PM10_dataframes, ignore_index=True, sort=False)
df_final_PM25 = pd.concat(PM25_dataframes, ignore_index=True, sort=False)
df_final_TMP = pd.concat(TMP_dataframes, ignore_index=True, sort=False)
df_final_WDR = pd.concat(WDR_dataframes, ignore_index=True, sort=False)
df_final_WSP = pd.concat(WSP_dataframes, ignore_index=True, sort=False)
df_final_RH = pd.concat(RH_dataframes, ignore_index=True, sort=False)


file_name_CO = os.path.join(files_path,"CO.csv")
file_name_O3 = os.path.join(files_path,"O3.csv")
file_name_NO = os.path.join(files_path,"NO.csv")
file_name_NO2 = os.path.join(files_path,"NO2.csv")
file_name_NOX = os.path.join(files_path,"NOX.csv")
file_name_PM10 = os.path.join(files_path,"PM10.csv")
file_name_PM25 = os.path.join(files_path,"PM25.csv")
file_name_TMP = os.path.join(files_path,"TMP.csv")
file_name_WDR = os.path.join(files_path,"WDR.csv")
file_name_WSP = os.path.join(files_path,"WSP.csv")
file_name_RH = os.path.join(files_path,"RH.csv")
file_name_SO2 = os.path.join(files_path,"SO2.csv")

df_final_CO.to_csv(file_name_CO, index=False)
df_final_SO2.to_csv(file_name_SO2, index=False)
df_final_NO.to_csv(file_name_NO, index=False)
df_final_NO2.to_csv(file_name_NO2, index=False)
df_final_NOX.to_csv(file_name_NOX, index=False)
df_final_O3.to_csv(file_name_O3, index=False)
df_final_PM10.to_csv(file_name_PM10, index=False)
df_final_PM25.to_csv(file_name_PM25, index=False)
df_final_TMP.to_csv(file_name_TMP, index=False)
df_final_WSP.to_csv(file_name_WSP, index=False)
df_final_WDR.to_csv(file_name_WDR, index=False)
df_final_RH.to_csv(file_name_RH, index=False)


Se cambian los datos vacios con valor de "nr" por NaN, y se separa la fecha en columnas de día, mes, año, y se crea una nueva con los datos de la fecha y la hora para poder ordenarlos de forma ascendente

# Section 3

Se guarda toda la info de todos los contaminantes en un solo dataframe

In [10]:
dir = "Archivos_contaminantes_juntas"
dir2 = "Archivos_contaminantes_juntas2"
#final_filename = "data_31May_2024_all_stations.csv"
final_filename = "data_jun_sep_2024_all_stations.csv"
if not os.path.exists(dir2):
    os.makedirs(dir2)

files = os.listdir(dir)
dataframes = []
for file_name in files:
    file_dir = os.path.join(dir, file_name)
    df = pd.read_csv(file_dir, low_memory=False)
    df.replace('nr', np.nan, inplace=True)
    columns = df.columns
    for column in columns:
        if 'Unnamed' in column:
            df = df.drop(column, axis=1)
    df['Fecha'] = df['Fecha'].str.replace('/', '-')
    df[['dia', 'mes', 'year']] = df['Fecha'].str.split('-', expand=True)     #separamos la fecha para poder unirla con la hora
    df['Hora'] = df['Hora'].replace(24, 0)  #cambiamos las 24 horas por las 0 horas
    # Convertir la columna 'horas' al formato de tiempo de Pandas
    df['Hora'] = pd.to_datetime(df['Hora'], unit='h')
    # Formatear la columna 'horas' en el formato deseado
    df['Hora'] = df['Hora'].dt.strftime('%H:%M')
    df["fecha_hora"] = df['year'] + "/"+df['mes'] +"/"+df["dia"]+" "+df["Hora"]
    df= df.sort_values(by='fecha_hora') 
    
    df["pollutant"] = file_name.split(".")[0]
    columna_especifica="fecha_hora"
    # Verificar si la columna tiene valores NaN
    tiene_nan = df[columna_especifica].isnull().any()

    # Mostrar el resultado
    if tiene_nan:
        print(f"La columna '{columna_especifica}' tiene valores NaN.")
    else:
        print(f"La columna '{columna_especifica}' no tiene valores NaN.")

    file_dir2 = os.path.join(dir2, file_name)
    df.to_csv(file_dir2, index=False) 
    dataframes.append(df)
df_final = pd.concat(dataframes, ignore_index=False, sort=False)
df_final.to_csv(final_filename, index=False)
df = pd.read_csv(final_filename)
del df["LVI"]
del df["SS1"]
if "Unnamed: 0" in df.keys():
    del df["Unnamed: 0"]
df.to_csv(final_filename , index=False)
df

La columna 'fecha_hora' no tiene valores NaN.
La columna 'fecha_hora' no tiene valores NaN.
La columna 'fecha_hora' no tiene valores NaN.
La columna 'fecha_hora' no tiene valores NaN.
La columna 'fecha_hora' no tiene valores NaN.
La columna 'fecha_hora' no tiene valores NaN.
La columna 'fecha_hora' no tiene valores NaN.
La columna 'fecha_hora' no tiene valores NaN.
La columna 'fecha_hora' no tiene valores NaN.
La columna 'fecha_hora' no tiene valores NaN.
La columna 'fecha_hora' no tiene valores NaN.
La columna 'fecha_hora' no tiene valores NaN.


,Fecha,Hora,ACO,AJM,AJU,ARA,ATI,AZC,BJU,CAM,...,UAX,UIZ,VAL,VIF,XAL,dia,mes,year,fecha_hora,pollutant
0,01-06-2024,00:00,NaN,0.14,NaN,NaN,0.76,NaN,0.63,NaN,...,NaN,0.58,NaN,0.80,0.91,1,6,2024,2024/06/01 00:00,CO
1,01-06-2024,01:00,NaN,0.30,NaN,NaN,0.67,NaN,0.77,NaN,...,NaN,1.14,NaN,0.70,0.58,1,6,2024,2024/06/01 01:00,CO
2,01-06-2024,02:00,NaN,0.20,NaN,NaN,0.54,NaN,0.64,NaN,...,NaN,0.80,NaN,0.61,0.99,1,6,2024,2024/06/01 02:00,CO
3,01-06-2024,03:00,NaN,0.14,NaN,NaN,0.61,NaN,0.64,NaN,...,NaN,0.72,NaN,0.71,0.89,1,6,2024,2024/06/01 03:00,CO
4,01-06-2024,04:00,NaN,0.20,NaN,NaN,0.60,NaN,0.55,NaN,...,NaN,1.07,NaN,0.73,0.74,1,6,2024,2024/06/01 04:00,CO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35047,30-09-2024,19:00,3.8,4.30,1.9,NaN,NaN,NaN,2.90,NaN,...,2.1,1.50,NaN,NaN,NaN,30,9,2024,2024/09/30 19:00,WSP
35048,30-09-2024,20:00,2.5,2.50,2.0,NaN,NaN,NaN,1.30,NaN,...,1.7,1.30,NaN,NaN,NaN,30,9,2024,2024/09/30 20:00,WSP
35049,30-09-2024,21:00,2.8,2.30,2.2,NaN,NaN,NaN,2.20,NaN,...,2.2,1.90,NaN,NaN,NaN,30,9,2024,2024/09/30 21:00,WSP
35050,30-09-2024,22:00,2.3,2.80,1.7,NaN,NaN,NaN,2.50,NaN,...,2.5,2.30,NaN,NaN,NaN,30,9,2024,2024/09/30 22:00,WSP


In [15]:
df.keys()

Index(['Fecha', 'Hora', 'ACO', 'AJM', 'AJU', 'ARA', 'ATI', 'AZC', 'BJU', 'CAM',
       'CCA', 'CES', 'MON', 'CHO', 'COY', 'CUA', 'CUT', 'FAC', 'FAR', 'GAM',
       'HGM', 'IMP', 'INN', 'IZT', 'LAG', 'LLA', 'LPR', 'MER', 'MGH', 'MPA',
       'NEZ', 'PED', 'PLA', 'SAC', 'SAG', 'SFE', 'SJA', 'SUR', 'TAC', 'TAH',
       'TAX', 'TEC', 'TLA', 'TLI', 'TPN', 'UAX', 'UIZ', 'VAL', 'VIF', 'XAL',
       'dia', 'mes', 'year', 'fecha_hora', 'pollutant'],
      dtype='object')

Número de estaciones de las que se tiene información

In [11]:
len(df.keys())-7

48

# Section 4

## No ejecutar si se hace desde cero (es decir, no hay información antigua con la que juntar la nueva)
Unir información nueva con antigua

In [17]:
old_df = pd.read_csv("data_15May_2024_all_stations.csv")
old_df["fecha_hora"]

0          2005/01/01 00:00
1          2005/01/01 01:00
2          2005/01/01 02:00
3          2005/01/01 03:00
4          2005/01/01 04:00
                 ...       
2015452    2024/05/15 13:00
2015453    2024/05/15 14:00
2015454    2024/05/15 15:00
2015455    2024/05/15 16:00
2015456    2024/05/15 17:00
Name: fecha_hora, Length: 2015457, dtype: object

In [21]:
dir = "data_31May_2024_all_stations.csv"
dir = final_filename
df = pd.read_csv(dir)
df["fecha_hora"]

0       2024/05/01 00:00
1       2024/05/01 01:00
2       2024/05/01 02:00
3       2024/05/01 03:00
4       2024/05/01 04:00
              ...       
8923    2024/05/31 19:00
8924    2024/05/31 20:00
8925    2024/05/31 21:00
8926    2024/05/31 22:00
8927    2024/05/31 23:00
Name: fecha_hora, Length: 8928, dtype: object

In [22]:
# Ejecutar solo si se tiene información de la parte restante de un mes que ya estaba
# Hay que borrar los datos que ya se tienen de ese mes

#dir = "data_31May_2024_all_stations.csv"
dir = final_filename
df = pd.read_csv(dir, dtype={'fecha_hora': 'str'})
# Convierte la columna 'fecha' a tipo timestamp
df['fecha_hora'] = pd.to_datetime(df['fecha_hora'])
df['fecha_hora'] = pd.to_datetime(df['fecha_hora'], format='%d/%m/%Y')
#df['fecha_hora'] = pd.to_datetime(df['fecha_hora'], format='%Y/%m/%d %H:%M:%S')
fecha_limite = pd.Timestamp('2024/05/15 17:00')
new_set = df[df['fecha_hora'] > fecha_limite].copy()
new_set['fecha_hora'] = new_set['fecha_hora'].dt.strftime('%Y/%m/%d %H:%M')
new_set["fecha_hora"]
new_set.to_csv(dir, index=False)

In [23]:
len(old_df)+len(new_set)

2020137

In [24]:
df_concat = pd.concat([old_df, new_set])
print(len(df_concat))
df_concat.to_csv(final_filename)

2020137


# Section 5

In [12]:
#final_filename = "all_data_2005xstation_31_05_2024.csv"
#final_filename = "data_31May_2024_all_stations.csv"
final_filename = "data_jun_sep_2024_all_stations.csv"
df = pd.read_csv(final_filename)
estaciones = df.keys()
borrar = ['Fecha', 'Hora', 'dia', 'mes', 'year', 'fecha_hora', 'pollutant']
estaciones = [elemento for elemento in estaciones if elemento not in borrar]
len(estaciones)

48

Revisar los nombres de las carpetas donde se guardarán y de donde se tomarán los datos

In [13]:
#dir_to_save = "datos_por_estacion"
dir_to_save = "datos_por_estacion_nuevos"
#dir_to_save = "all_data_2005xstation_31_05_2024"
if not os.path.exists(dir_to_save):
    os.makedirs(dir_to_save)
    

pollutants = ["CO", "NO", "NOX", "NO2", "O3", "PM10", "PM25", "RH", "SO2", "TMP", "WDR", "WSP"]

for estacion in estaciones:
  columnas_interes = ["fecha_hora", "pollutant"]
  columnas_interes.append(estacion)
  datos_modelos = df[columnas_interes]
  datos_modelos = datos_modelos.rename(columns={estacion: "value"})
  dataframes = []
  for pollutant in pollutants:
    data = datos_modelos[datos_modelos.pollutant==pollutant]
    data = data.set_index("fecha_hora")
    data.index.name = 'date'
    data.drop('pollutant', axis=1, inplace=True)
    data = data.rename(columns={"value":pollutant})
    dataframes.append(data)

  df_merged = pd.DataFrame()
  columnas = ['date']
  df_merged = pd.DataFrame(columns=columnas)
  for dataframe in dataframes:
    df_merged = pd.merge(df_merged, dataframe, on='date', how='outer')

  df_merged = df_merged.reset_index()
  df_merged = df_merged.drop(columns=["index"])
  df_merged_sin_fecha = df_merged.drop(columns=["date"])
  indices_filas_vacias = df_merged_sin_fecha[df_merged_sin_fecha.isna().all(axis=1)].index
  df_filtrado = df_merged.drop(indices_filas_vacias)
  df_filtrado = df_filtrado.set_index("date")
  df_filtrado= df_filtrado.sort_values(by='date') 
  df_filtrado = df_filtrado.reset_index()
  try:
    df_filtrado['date'] = df_filtrado['date'].dt.strftime('%Y/%m/%d %H:%M')
  #"""
  except:
    for i in range(len(df_filtrado)):
      try: 
        fecha = df_filtrado.iloc[i]['date']
        if type(fecha) == str: 
                if len(fecha) == 19:
                    # Cambiar el formato de la fecha a 'YYYY/MM/DD HH:MM'
                    fecha_dt = datetime.datetime.strptime(fecha, '%Y/%m/%d %H:%M:%S')
                    # Formatear la fecha al formato deseado (sin segundos)
                    df_filtrado.iloc[i]['date'] = fecha_dt.strftime('%Y/%m/%d %H:%M')
                elif len(fecha) == 16:
                    #Ya esta con el formato 'YYYY/MM/DD HH:MM'
                    df_filtrado.iloc[i]['date'] = pd.to_datetime(fecha, format='%Y/%m/%d %H:%M')
      except Exception as e:
        print(e)
        print(i,  type(df_filtrado.iloc[i]['date']))
        print(df_filtrado.iloc[i])
  #"""
  name = estacion + ".csv"
  dir_file = os.path.join(dir_to_save,name)
  df_filtrado.to_csv(dir_file, index=False)

C:\Users\valer\AppData\Local\Temp\ipykernel_20072\1070152861.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado.iloc[i]['date'] = pd.to_datetime(fecha, format='%Y/%m/%d %H:%M')
C:\Users\valer\AppData\Local\Temp\ipykernel_20072\1070152861.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado.iloc[i]['date'] = pd.to_datetime(fecha, format='%Y/%m/%d %H:%M')
C:\Users\valer\AppData\Local\Temp\ipykernel_20072\1070152861.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

Se borran las carpetan que ya no nos sirven

In [28]:
dir = "Archivos_contaminantes_juntas"
shutil.rmtree(dir)
dir = "Archivos_contaminantes_juntas2"
shutil.rmtree(dir)
#dir = "datos_por_estacion"
#shutil.rmtree(dir)

#os.remove(final_filename)